[View in Colaboratory](https://colab.research.google.com/github/sungreong/GAN/blob/master/LSGAN__MNIST_KERAS.ipynb)

In [0]:
from __future__ import print_function, division

In [0]:
!pip install -q keras

In [0]:
from keras.datasets import mnist
from keras.layers import Input , Dense, Reshape , Flatten, Dropout
from keras.layers import BatchNormalization ,Activation , ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D , Conv2D
from keras.models import Sequential , Model
from keras.optimizers import Adam

In [0]:
import matplotlib.pyplot as plt

In [0]:
import sys

In [0]:
import numpy as np

![대체 텍스트](https://preview.ibb.co/mQDVTT/image.png)

In [0]:
class LSGAN() :
    def __init__(self) :
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows , self.img_cols , self.channels)
        self.latent_dim = 100
        
        optimizer =Adam(0.0002, 0.5)
        
        # build and compile discriminator
        
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss="mse", optimizer=optimizer , metrics=["accuracy"])
        
        # build the generator
        self.generator = self.build_generator()
        
        z= Input(shape=(100,))
        img = self.generator(z)
        
        # 오직 generator만 훈련 시킬 것이다
        
        self.discriminator.trainable= False
        
        # generator 에서 생성된 이미지를 받아서 validity를 결정한다.
        valid = self.discriminator(img)
        
        # 결합된 모델 (stacked generator and discriminator)
        # discriminator를 멍청하게 만든느 학습을 훈련시킨다.
        
       
        self.combined = Model(z, valid)
        
        ###### 크로스엔트로피 대신 mse를 이용한  optimizer 한다.
        ## LSGAN 핵심 부분 
        self.combined.compile(loss="mse", optimizer=optimizer)
        
    def build_generator(self) :
        
        model= Sequential()
        
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1024))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation="tanh")) # Dense(784, activation="tanh")
        model.add(Reshape(self.img_shape)) # (28, 28,1)
        
        model.summary()
        
        noise= Input(shape=(self.latent_dim , ))
        
        img= model(noise)
        
        return Model(noise, img)
    
    def build_discriminator(self) :
        
        
        model = Sequential()
        
        model.add(Flatten(input_shape=self.img_shape)) #(28, 28, 1)
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.02))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.02))
        ## no softmax !! mse 사용중
        model.add(Dense(1))
        model.summary()
        
        img = Input(shape=self.img_shape)
        validity= model(img)
        
        return (Model(img,validity))
    
    def train(self, epochs, batch_size = 128 , sample_interval = 50) :
        
        (X_train, _), (_, _) =mnist.load_data()
        
        X_train = (X_train.astype(np.float32)- 127.5)/ 127.5
        X_train = np.expand_dims(X_train, axis=3) ## (60000, 28, 28, 1)
        
        valid = np.ones((batch_size, 1))
        fake= np.zeros((batch_size, 1)) # (128, 1)
        
        for epoch in range(epochs) :
            
            
            ### train discriminator
            
            
            idx = np.random.randint(0, X_train.shape[0], batch_size) # 0~60000 에서 128 개 뽑기 
            imgs =X_train[idx]
            
            noise = np.random.normal(0,1,(batch_size, self.latent_dim)) # (128, 100)
            
            gen_imgs= self.generator.predict(noise)
            
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5* np.add(d_loss_real, d_loss_fake)
            
            ### train generator
            for _ in range(4):
                g_loss = self.combined.train_on_batch(noise, valid)
            
            print("%d [D loss : %f, acc. : %.2f%%] [G loss : %f]" %(epoch, d_loss[0], 100*d_loss[1], g_loss))
            
            
            # 간격마다 샘플이미지 생성하기 
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()            
            
        
        

In [41]:
if __name__ == '__main__':
    gan = LSGAN()
    gan.train(epochs=10000, batch_size=32, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_10 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_56 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_33 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_57 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_34 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_58 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss : 0.590725, acc. : 57.81%] [G loss : 0.071978]
1 [D loss : 1.498449, acc. : 39.06%] [G loss : 0.057615]
2 [D loss : 0.585058, acc. : 48.44%] [G loss : 0.031005]
3 [D loss : 0.215251, acc. : 76.56%] [G loss : 0.051202]
4 [D loss : 0.220278, acc. : 79.69%] [G loss : 0.045592]
5 [D loss : 0.185337, acc. : 68.75%] [G loss : 0.049270]
6 [D loss : 0.122098, acc. : 87.50%] [G loss : 0.042761]
7 [D loss : 0.136113, acc. : 84.38%] [G loss : 0.046616]
8 [D loss : 0.113510, acc. : 79.69%] [G loss : 0.036575]
9 [D loss : 0.160616, acc. : 75.00%] [G loss : 0.045724]
10 [D loss : 0.279819, acc. : 64.06%] [G loss : 0.044798]
11 [D loss : 0.395767, acc. : 48.44%] [G loss : 0.048613]
12 [D loss : 0.617998, acc. : 40.62%] [G loss : 0.037969]
13 [D loss : 0.416179, acc. : 45.31%] [G loss : 0.040797]
14 [D loss : 0.184599, acc. : 71.88%] [G loss : 0.034505]
15 [D loss : 0.151476, acc. : 78.12%] [G loss : 0.043802]
16 [D loss : 0.092222, acc. : 92.19%] [G loss : 0.040245]
17 [D loss : 0.092094, a

112 [D loss : 0.059179, acc. : 98.44%] [G loss : 0.063099]
113 [D loss : 0.084829, acc. : 92.19%] [G loss : 0.040691]
114 [D loss : 0.081144, acc. : 90.62%] [G loss : 0.061378]
115 [D loss : 0.051695, acc. : 95.31%] [G loss : 0.096803]
116 [D loss : 0.054020, acc. : 95.31%] [G loss : 0.062088]
117 [D loss : 0.049901, acc. : 93.75%] [G loss : 0.057587]
118 [D loss : 0.049048, acc. : 95.31%] [G loss : 0.058028]
119 [D loss : 0.052893, acc. : 95.31%] [G loss : 0.047449]
120 [D loss : 0.048903, acc. : 96.88%] [G loss : 0.072739]
121 [D loss : 0.062018, acc. : 93.75%] [G loss : 0.084551]
122 [D loss : 0.050993, acc. : 95.31%] [G loss : 0.096779]
123 [D loss : 0.085455, acc. : 95.31%] [G loss : 0.054207]
124 [D loss : 0.058171, acc. : 96.88%] [G loss : 0.099572]
125 [D loss : 0.096270, acc. : 93.75%] [G loss : 0.093769]
126 [D loss : 0.066905, acc. : 95.31%] [G loss : 0.119683]
127 [D loss : 0.074599, acc. : 90.62%] [G loss : 0.080468]
128 [D loss : 0.079058, acc. : 95.31%] [G loss : 0.13766

224 [D loss : 0.239714, acc. : 57.81%] [G loss : 0.298704]
225 [D loss : 0.187853, acc. : 75.00%] [G loss : 0.263613]
226 [D loss : 0.189261, acc. : 65.62%] [G loss : 0.368294]
227 [D loss : 0.247275, acc. : 64.06%] [G loss : 0.247916]
228 [D loss : 0.121124, acc. : 79.69%] [G loss : 0.251432]
229 [D loss : 0.166677, acc. : 68.75%] [G loss : 0.364537]
230 [D loss : 0.206738, acc. : 56.25%] [G loss : 0.291030]
231 [D loss : 0.145348, acc. : 79.69%] [G loss : 0.300790]
232 [D loss : 0.186782, acc. : 67.19%] [G loss : 0.313545]
233 [D loss : 0.193331, acc. : 70.31%] [G loss : 0.256544]
234 [D loss : 0.187591, acc. : 62.50%] [G loss : 0.411776]
235 [D loss : 0.254035, acc. : 48.44%] [G loss : 0.282055]
236 [D loss : 0.151748, acc. : 79.69%] [G loss : 0.256478]
237 [D loss : 0.212664, acc. : 62.50%] [G loss : 0.354798]
238 [D loss : 0.242180, acc. : 54.69%] [G loss : 0.299010]
239 [D loss : 0.187921, acc. : 64.06%] [G loss : 0.336129]
240 [D loss : 0.260978, acc. : 54.69%] [G loss : 0.27017

...

9510 [D loss : 0.168113, acc. : 76.56%] [G loss : 0.250977]
9511 [D loss : 0.180109, acc. : 78.12%] [G loss : 0.261712]
9512 [D loss : 0.204287, acc. : 60.94%] [G loss : 0.239958]
9513 [D loss : 0.192240, acc. : 70.31%] [G loss : 0.204748]
9514 [D loss : 0.197124, acc. : 68.75%] [G loss : 0.258875]
9515 [D loss : 0.185429, acc. : 71.88%] [G loss : 0.232664]
9516 [D loss : 0.191894, acc. : 75.00%] [G loss : 0.182605]
9517 [D loss : 0.224149, acc. : 64.06%] [G loss : 0.191172]
9518 [D loss : 0.246411, acc. : 57.81%] [G loss : 0.191267]
9519 [D loss : 0.177970, acc. : 76.56%] [G loss : 0.228546]
9520 [D loss : 0.213656, acc. : 68.75%] [G loss : 0.249347]
9521 [D loss : 0.215092, acc. : 64.06%] [G loss : 0.244476]
9522 [D loss : 0.199334, acc. : 67.19%] [G loss : 0.244870]
9523 [D loss : 0.183619, acc. : 73.44%] [G loss : 0.203258]
9524 [D loss : 0.224957, acc. : 65.62%] [G loss : 0.210140]
9525 [D loss : 0.211836, acc. : 64.06%] [G loss : 0.229259]
9526 [D loss : 0.184025, acc. : 70.31%] 

9622 [D loss : 0.185556, acc. : 73.44%] [G loss : 0.208553]
9623 [D loss : 0.196658, acc. : 71.88%] [G loss : 0.209525]
9624 [D loss : 0.140258, acc. : 84.38%] [G loss : 0.281878]
9625 [D loss : 0.205888, acc. : 73.44%] [G loss : 0.232568]
9626 [D loss : 0.167700, acc. : 79.69%] [G loss : 0.237049]
9627 [D loss : 0.202805, acc. : 67.19%] [G loss : 0.205109]
9628 [D loss : 0.185989, acc. : 75.00%] [G loss : 0.230268]
9629 [D loss : 0.244165, acc. : 51.56%] [G loss : 0.230645]
9630 [D loss : 0.186955, acc. : 68.75%] [G loss : 0.221025]
9631 [D loss : 0.214263, acc. : 71.88%] [G loss : 0.208178]
9632 [D loss : 0.201063, acc. : 71.88%] [G loss : 0.224709]
9633 [D loss : 0.198801, acc. : 68.75%] [G loss : 0.219211]
9634 [D loss : 0.189904, acc. : 76.56%] [G loss : 0.250230]
9635 [D loss : 0.218773, acc. : 54.69%] [G loss : 0.238617]
9636 [D loss : 0.215040, acc. : 65.62%] [G loss : 0.233385]
9637 [D loss : 0.198982, acc. : 67.19%] [G loss : 0.249373]
9638 [D loss : 0.184472, acc. : 71.88%] 

9734 [D loss : 0.231960, acc. : 62.50%] [G loss : 0.252838]
9735 [D loss : 0.205197, acc. : 70.31%] [G loss : 0.275186]
9736 [D loss : 0.199205, acc. : 73.44%] [G loss : 0.243656]
9737 [D loss : 0.224660, acc. : 60.94%] [G loss : 0.219985]
9738 [D loss : 0.223550, acc. : 64.06%] [G loss : 0.196682]
9739 [D loss : 0.140957, acc. : 85.94%] [G loss : 0.257753]
9740 [D loss : 0.191639, acc. : 73.44%] [G loss : 0.238856]
9741 [D loss : 0.205121, acc. : 65.62%] [G loss : 0.247018]
9742 [D loss : 0.197447, acc. : 70.31%] [G loss : 0.248265]
9743 [D loss : 0.207545, acc. : 75.00%] [G loss : 0.236783]
9744 [D loss : 0.213617, acc. : 65.62%] [G loss : 0.261674]
9745 [D loss : 0.197899, acc. : 70.31%] [G loss : 0.264422]
9746 [D loss : 0.211545, acc. : 65.62%] [G loss : 0.224354]
9747 [D loss : 0.196304, acc. : 70.31%] [G loss : 0.235787]
9748 [D loss : 0.196959, acc. : 73.44%] [G loss : 0.211200]
9749 [D loss : 0.190977, acc. : 67.19%] [G loss : 0.206155]
9750 [D loss : 0.182182, acc. : 76.56%] 

9846 [D loss : 0.206790, acc. : 68.75%] [G loss : 0.215777]
9847 [D loss : 0.223808, acc. : 67.19%] [G loss : 0.266544]
9848 [D loss : 0.177847, acc. : 73.44%] [G loss : 0.219742]
9849 [D loss : 0.191057, acc. : 71.88%] [G loss : 0.213587]
9850 [D loss : 0.175932, acc. : 67.19%] [G loss : 0.252468]
9851 [D loss : 0.252237, acc. : 60.94%] [G loss : 0.227553]
9852 [D loss : 0.217256, acc. : 65.62%] [G loss : 0.204343]
9853 [D loss : 0.184776, acc. : 68.75%] [G loss : 0.260092]
9854 [D loss : 0.205777, acc. : 65.62%] [G loss : 0.248585]
9855 [D loss : 0.234168, acc. : 60.94%] [G loss : 0.239174]
9856 [D loss : 0.214730, acc. : 68.75%] [G loss : 0.199771]
9857 [D loss : 0.194387, acc. : 70.31%] [G loss : 0.258897]
9858 [D loss : 0.220680, acc. : 64.06%] [G loss : 0.227777]
9859 [D loss : 0.225791, acc. : 68.75%] [G loss : 0.220969]
9860 [D loss : 0.221492, acc. : 57.81%] [G loss : 0.230951]
9861 [D loss : 0.157131, acc. : 81.25%] [G loss : 0.292180]
9862 [D loss : 0.213432, acc. : 68.75%] 

9957 [D loss : 0.205765, acc. : 68.75%] [G loss : 0.235543]
9958 [D loss : 0.225569, acc. : 57.81%] [G loss : 0.208092]
9959 [D loss : 0.207700, acc. : 64.06%] [G loss : 0.197032]
9960 [D loss : 0.228638, acc. : 64.06%] [G loss : 0.208383]
9961 [D loss : 0.221557, acc. : 60.94%] [G loss : 0.215086]
9962 [D loss : 0.211694, acc. : 67.19%] [G loss : 0.185924]
9963 [D loss : 0.239927, acc. : 56.25%] [G loss : 0.225470]
9964 [D loss : 0.191141, acc. : 70.31%] [G loss : 0.246899]
9965 [D loss : 0.194421, acc. : 70.31%] [G loss : 0.228225]
9966 [D loss : 0.192638, acc. : 67.19%] [G loss : 0.236905]
9967 [D loss : 0.194296, acc. : 67.19%] [G loss : 0.214007]
9968 [D loss : 0.204888, acc. : 65.62%] [G loss : 0.214375]
9969 [D loss : 0.206159, acc. : 64.06%] [G loss : 0.223085]
9970 [D loss : 0.217030, acc. : 64.06%] [G loss : 0.247603]
9971 [D loss : 0.226776, acc. : 65.62%] [G loss : 0.255416]
9972 [D loss : 0.187269, acc. : 67.19%] [G loss : 0.223861]
9973 [D loss : 0.195251, acc. : 68.75%] 

In [30]:
(X_train, _), (_, _) =mnist.load_data()
np.unique((X_train.astype(np.float32)- 127.5))
print(X_train.shape)
X_train = np.expand_dims(X_train, axis=3)
print(X_train.shape)

(60000, 28, 28)
(60000, 28, 28, 1)


In [38]:
np.prod((28,28,1))

784

8